In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("./gdrive/MyDrive/challenges/DHCP.csv")

In [ ]:
print("="*30)
print(data.columns)
print("="*30)
print(data.head())
print("="*30)
print(data.dtypes)

In [ ]:
data.describe()

In [ ]:
print(data.isnull().sum() / data.shape[0])
print("="*50)
print(data.isnull().sum())

In [ ]:
print(data)

In [ ]:
import seaborn as sns

heatmap_data = data[['Svr_detect', 'Svr_connect', 'Ss_request', 'Ss_Established']]

colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})


In [ ]:
data.corr()

In [ ]:
from datetime import datetime

data['Timestamp'] = data.Timestamp.str.split('-').str[0]
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y%m%d_%H%M')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
df_data = data.columns.drop('Timestamp')

In [ ]:
plt.style.use('ggplot')

plt.figure(figsize=(30, 10))
plt.suptitle("columns data scatter plot", fontsize=20)

# data.loc[:, data.columns != 'Timestamp'].columns

for idx, col_name in enumerate(data.loc[:, data.columns != 'Timestamp'].columns):
  plt.subplot(2, 2, idx + 1)
  plt.title(f"{col_name} scatter plot")
  plt.scatter(data['Timestamp'], data[col_name])
  plt.xlabel(f"{col_name}")

plt.show()

In [ ]:
import numpy as np

def normalization(df):
  return df.apply(lambda value: ((value - np.mean(value)) / np.std(value)))

In [ ]:
df_data_cols = data.columns.drop(['Timestamp'])
df_train = normalization(data[df_data_cols])

print(df_train)

In [ ]:
plt.style.use('ggplot')

plt.figure(figsize=(30, 10))
plt.suptitle("normalization columns data scatter plot", fontsize=20)

print(df_train)

for idx, col_name in enumerate(df_train):
  plt.subplot(2, 2, idx + 1)
  plt.title(f"{col_name} scatter plot")
  plt.scatter(data['Timestamp'], df_train[col_name])
  plt.xlabel(f"{col_name}")

In [ ]:
plt.figure(figsize=(22, 8))

plt.style.use('ggplot')

plt.figure(figsize=(30, 10))
plt.suptitle("normalization columns data scatter plot", fontsize=20)

print(df_train)

for idx, col_name in enumerate(df_train):
  plt.subplot(2, 2, idx + 1)
  plt.title(f"{col_name} scatter plot")
  plt.plot(data['Timestamp'], df_train[col_name])
  plt.xlabel(f"{col_name}")

plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split

# Svr_detect  Svr_connect  Ss_request  Ss_Established
train_set, test_set = train_test_split(df_train, test_size=0.2, random_state=2022)

# print(train_set, test_set)
# model = ARIMA(data, order=(2, 1, 2))
# model_fit = model.fit(disp=0)
timeSeries = data.loc[:, ["Timestamp", "Svr_detect"]]
timeSeries.index = timeSeries.Timestamp

#timeSeries.fillna()
ts = timeSeries.drop("Timestamp", axis = 1)
print(ts['Svr_detect'].value_counts())

ts.loc[ts.Svr_detect.isnull(),'Svr_detect'] = 0.0
print(ts.Svr_detect.isnull().sum())

from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(ts['Svr_detect'], model='additive', freq=7)

fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20, 15)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pandas import datetime

model = ARIMA(ts, order=(2, 1, 2))
model_fit = model.fit(disp=0)

start_index = datetime(1944, 6, 25)

model = ARIMA